# Simple Hash

simple modulo hash function:
$$
    f(x) = x\mod c\text{, where }x \geq 0\text{, and }c = 2^{32} - 1.
$$

In [ ]:
import math
import struct

c = 2**32 - 1

class Hash_value:
    def __init__(self, hashvalue):
        self.bits = format(hashvalue,'b').zfill(32)
        self.integer =  hashvalue

def simple_digest(m):
  if isinstance(m, str) and all(c in '01' for c in m):  # If input is a bit string
    m = int(m,2)
  elif not isinstance(m, int):
    raise ValueError("Input must be a bit string or integer")
  return Hash_value((m + 8) % c)


print(simple_digest(123456789).bits)
print(simple_digest(123456789).integer)

00000111010110111100110100011101
123456797


# Generate Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to Google Drive
file_path = '/content/drive/MyDrive/Datasets/'

Mounted at /content/drive


## Dataset for FFN
- Input: hash as normed integer
- Output: message as bitvector

In [ ]:
import numpy as np
import random
import struct

def H(m):
    return simple_digest(m)

def generate_bitstring(length):
    return format(random.randint(0,c - 1),'b').zfill(104)

def generate_random_bitstrings(num_samples, bitlength):
    bitstrings = set()
    while(len(bitstrings) < num_samples):
        bitstring = generate_bitstring(bitlength)
        bitstrings.add(bitstring)
    return bitstrings

def generate_dataset(num_samples=100000, msglength = 104):# 104 bit messages are processed in one block
    X = []  # Input (normalized Hashvalues)
    Y_int = []
    Y = []  # Output (128-Bit-Bitvectors)
    msgs = generate_random_bitstrings(num_samples, msglength)
    for msg in msgs:
        hash = H(msg)  # calculate 32-Bit-Hash
        hash_normalized = hash.integer / (2**32 - 1)  # Normalized to [0,1]
        msg_bits = np.array(list(msg), dtype=np.uint8)  # 128 Bit

        X.append([hash_normalized])
        Y.append(msg_bits)
        Y_int.append([int(msg,2)/c])

    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)
    Y_int = np.array(Y_int, dtype=np.float32)

    np.save(f"{file_path}X_FFN_simpleHash.npy", X)
    np.save(f"{file_path}Y_FFN_simpleHash.npy", Y)
    np.save(f"{file_path}Y_int_FFN_simpleHash.npy", Y_int)


generate_dataset(100000)

## Dataset for CNN
- Features: hash as bitvector
- Label: message as bitvector

In [ ]:
import numpy as np
import random
import struct

# MD5 Light, returning integer
def H(m) -> str:
    return simple_digest(m).bits

def generate_bitstring(length):
    return ''.join(random.choice('01') for _ in range(length))

def generate_random_bitstrings(num_samples, bitlength):
    bitstrings = set()
    while(len(bitstrings) < num_samples):
        bitstring = generate_bitstring(bitlength)
        bitstrings.add(bitstring)
    return bitstrings

def generate_dataset(num_samples=100000, msglength = 104):# 104 bit messages are processed in one block
    X = []  # Input (32-Bit-Vectors)
    Y = []  # Output (128-Bit-Bitvectors)
    msgs = generate_random_bitstrings(num_samples, msglength)
    for msg in msgs:
        hash = H(msg)  # calculate 32-Bit-Hash
        hash_bits = np.array(list(hash), dtype=np.uint8)  # 32 Bit
        msg_bits = np.array(list(msg), dtype=np.uint8)  # 128 Bit

        X.append(hash_bits)
        Y.append(msg_bits)

    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)

    np.save(f"{file_path}X_CNN_simpleHash.npy", X)
    np.save(f"{file_path}Y_CNN_simpleHash.npy", Y)


generate_dataset(100000)

In [ ]:
!pip install scikeras scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.4 MB/s eta 0:00:00


# Feedforward Neural Network Based Pre Image Attack on MD5 Light


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2

In [ ]:
# load Dataset
X = np.load(f"{file_path}X_FFN_simpleHash.npy")  # Normalisierte Hashwerte
Y = np.load(f"{file_path}Y_FFN_simpleHash.npy")  # 104-Bit-Nachrichten als Bitvektoren
Y_int = np.load(f"{file_path}Y_int_FFN_simpleHash.npy")  # 104-Bit-Nachrichten als Integer

# Überprüfen der Datenform
print(f"X Shape: {X.shape}")  # (100000, 1)
print(f"Y Shape: {Y.shape}")  # (100000, 104)
print(f"Y_int Shape: {Y_int.shape}")  # (100000, 104)

# 80% Training, 20% Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_int, test_size=0.2, random_state=42)

X Shape: (100000, 1)
Y Shape: (100000, 104)
Y_int Shape: (100000, 1)


#### Model Architectur:
Since the function is periodic and simple, a small fully connected neural network with ReLU activation should work.

In [ ]:
# Modellaufbau
def create_model(learning_rate=0.005, neurons=[256,512,256], activation_fct='relu', dropout_rate=0.2):
  model = Sequential([
    Input(shape=(1,)), # Input Layer
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # Hidden layer
    #Dense(104, activation="sigmoid")  # 128 Neuronen, Sigmoid für bitweise Ausgabe
    Dense(1)  # Output layer
  ])

  model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["accuracy"])
  return model

model = create_model()
# Modellübersicht
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                     │ (None, 32)                  │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,241 (8.75 KB)

 Trainable params: 2,241 (8.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Modell trainieren
history = model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_split=0.1)

Epoch 1/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0160 - val_accuracy: 0.0000e+00 - val_loss: 5.6407e-04
Epoch 2/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0000e+00 - loss: 5.2053e-04 - val_accuracy: 0.0000e+00 - val_loss: 4.0229e-04
Epoch 3/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 4.1555e-04 - val_accuracy: 0.0000e+00 - val_loss: 3.5598e-04
Epoch 4/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0000e+00 - loss: 3.6304e-04 - val_accuracy: 0.0000e+00 - val_loss: 3.1918e-04
Epoch 5/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0000e+00 - loss: 3.3935e-04 - val_accuracy: 0.0000e+00 - val_loss: 2.9428e-04
Epoch 6/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0000e+00 - loss: 3.1358e-04 - val_accuracy: 0.0000e+00 - val_loss: 2.7496e-04
Epoch 7/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0000e+00 - loss: 3.1137e-04 - val_accuracy: 0.0000e+00 - val

## Bayesian Optimization

In [ ]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Daten laden
X = np.load(f"{file_path}X_FFN_MD5light.npy")
Y = np.load(f"{file_path}Y_FFN_MD5light.npy")

# Train-Test-Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

historys = []
params_ = []

# Ziel-Funktion für Optuna
def objective(trial):
    # Optimierbare Hyperparameter
    num_layers = trial.suggest_int("num_layers", 1, 5)
    neurons = trial.suggest_int("neurons", 128, 1024, step=128)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    batch_size = trial.suggest_int("batch_size", 32, 1024, step=32)
    activation = trial.suggest_categorical("activation", ["relu", "leaky_relu"])

    # Modell aufbauen
    model = Sequential()
    model.add(Input(shape=(1,)))
    model.add(Dense(neurons, activation=activation))

    for _ in range(num_layers - 1):
        model.add(Dense(neurons, activation=activation))

    model.add(Dense(104, activation="sigmoid"))  # Bitvektor als Ausgabe

    # Optimizer
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])

    # EarlyStopping Callback erstellen
    early_stopping = EarlyStopping(
      monitor='val_loss',  # Überwacht die Validierungs-Loss
      patience=10,          # Stoppt, wenn sich die Loss für 5 Epochen nicht verbessert
      restore_best_weights=True  # Stellt die besten Gewichte wieder her
    )
    history = model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, validation_split=0.1, verbose=0, callbacks = [early_stopping])
    historys.append(history)
    params_.append([num_layers,neurons,learning_rate,batch_size,activation])
    # Bewertung auf Testset
    loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    print(f"Loss: {loss}; Accuracy: {accuracy}")

    return 1 - loss  # Wir minimieren loss

# Bayesian Optimization starten
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Beste Hyperparameter ausgeben
print("Beste Hyperparameter:", study.best_params)


[I 2025-02-01 19:57:27,516] A new study created in memory with name: no-name-6e74af52-20e7-4280-a831-0bff2b93e8ba
<ipython-input-24-6803860465e4>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
[I 2025-02-01 19:57:51,895] Trial 0 finished with value: 0.0 and parameters: {'num_layers': 1, 'neurons': 512, 'learning_rate': 6.097150420496927e-05, 'batch_size': 608, 'activation': 'leaky_relu'}. Best is trial 0 with value: 0.0.


Loss: 0.6931487917900085; Accuracy: 0.0


[I 2025-02-01 19:58:50,457] Trial 1 finished with value: 4.999999873689376e-05 and parameters: {'num_layers': 3, 'neurons': 128, 'learning_rate': 1.2399758341628846e-05, 'batch_size': 160, 'activation': 'relu'}. Best is trial 1 with value: 4.999999873689376e-05.


Loss: 0.6931495666503906; Accuracy: 4.999999873689376e-05


[I 2025-02-01 20:07:04,972] Trial 2 finished with value: 0.020600000396370888 and parameters: {'num_layers': 5, 'neurons': 896, 'learning_rate': 0.00011485819454393576, 'batch_size': 928, 'activation': 'leaky_relu'}. Best is trial 2 with value: 0.020600000396370888.


...

Loss: 0.693181574344635; Accuracy: 0.023099999874830246


[I 2025-02-01 21:23:17,058] Trial 28 finished with value: 0.04194999858736992 and parameters: {'num_layers': 2, 'neurons': 1024, 'learning_rate': 0.0010905786942030234, 'batch_size': 832, 'activation': 'relu'}. Best is trial 9 with value: 0.07175000011920929.


Loss: 0.6931551098823547; Accuracy: 0.04194999858736992


[I 2025-02-01 21:29:04,560] Trial 29 finished with value: 0.0 and parameters: {'num_layers': 2, 'neurons': 1024, 'learning_rate': 0.002329401745700488, 'batch_size': 864, 'activation': 'relu'}. Best is trial 9 with value: 0.07175000011920929.


Loss: 0.6931600570678711; Accuracy: 0.0
Beste Hyperparameter: {'num_layers': 1, 'neurons': 384, 'learning_rate': 0.0009566255730068911, 'batch_size': 672, 'activation': 'relu'}


# Installation

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
